In [1]:
!nvidia-smi

Tue Jul 29 15:26:05 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:31:00.0 Off |                    0 |
| N/A   33C    P0              51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd
import ast
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import ast
import csv
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
DATASET_NAMES = [
                # "emotion", 
                # "math", 
                "mmlu", 
                # "programming"
]
DATASET_PATHS = [f"./datasets/{name}_processed.csv" for name in DATASET_NAMES]
# Load and concatenate all datasets
df = pd.concat([pd.read_csv(path, encoding="utf-8-sig") for path in DATASET_PATHS], ignore_index=True)

In [4]:
# ! df.dropna(inplace=True)    # for analysis and vis keep all  
df.head()

,type,prompt,category,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_0/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_1/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_2/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_3/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_4/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_5/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_6/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_7/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_8/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_9/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_10/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_11/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_12/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_13/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_14/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_15/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-top_mean_vals,gemma-scope-2b-pt-res-canonical-layer_16/width_16k/canonical-token_feature_ids,gemma-scope-2b-pt-res-canonical-layer_17/width_16k/canonical-top_mean_ids,gemma-scope-2b-pt-res-canonical-layer_17/width_16k/canonic

In [12]:
counts = df['type'].value_counts()
print(counts)

type
programming            5000
math                   4998
empathetic_dialogue    4992
mmlu                   4984
Name: count, dtype: int64


# Dot product within class

In [5]:
# create freq of vectors [0, 0, 0, .., 0]
VECTOR_SIZE_SAE = 16384
def to_vector(specialized_df, col_name, binary_vector=False):
    """
    input:
        specialized_df: pandas df 
        col_name: col name with its layer and type 
    output:
        vector of size 16500
            max neuron 16383
    """
    shared_neurons = [0] * VECTOR_SIZE_SAE
    for row_list_str in specialized_df[col_name]:      # TODO: optimize with counter
        # skip missing entries
        if pd.isna(row_list_str):
            continue
        neuron_list = ast.literal_eval(row_list_str)
        for neuron in neuron_list:
            if binary_vector:
                shared_neurons[neuron] = 1
            else:
                shared_neurons[neuron] += 1
            
    return shared_neurons
    
# to_vector(df, "gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids")
# first_list = ast.literal_eval(df["gemma-scope-2b-pt-res-canonical-layer0-token_feature_ids"].iloc[0] )  # now a Python list of ints

In [6]:
import numpy as np
import pandas as pd
import ast

def get_element_wise_product(typ, layer, class1, class2, vec1, vec2, top_n=10):
    """
    Identifies neurons that contribute most significantly to the dot product
    between two frequency vectors.

    Parameters:
    -----------
    typ : str
        A label for the type of comparison.
    layer : int or str
        Layer identifier.
    class1 : str
        Name of the first class.
    class2 : str
        Name of the second class.
    vec1, vec2 : list, np.array, or str
        The two frequency vectors (or their literal strings).
    top_n : int
        Number of top neurons to return.

    Returns:
    --------
    pd.DataFrame
        Columns = ['type', 'layer', 'class1', 'class2', 'Neuron_ID', 'Contribution']
        Sorted by descending Contribution, up to top_n rows.
    """
    # to numpy arrays
    v1 = np.array(vec1, dtype=float)
    v2 = np.array(vec2, dtype=float)

    # element-wise contributions
    contrib = v1 * v2

    # build full DataFrame
    df = pd.DataFrame({
        'type':       typ,
        'layer':      layer,
        'class1':     class1,
        'class2':     class2,
        'Neuron_ID':  np.arange(contrib.shape[0]),
        'Contribution': contrib
    })

    # keep only positive contributions
    df = df[df['Contribution'] > 0]

    # ensure all columns present even if empty
    return df.reindex(columns=[
        'type', 'layer', 'class1', 'class2', 'Neuron_ID', 'Contribution'
    ])


In [8]:
# DATASET_TYPES = {
#     "empathetic_dialogue": df.loc[df["type"] == "empathetic_dialogue", "category"].unique(),
#     "math":                df.loc[df["type"] == "math",                "category"].unique(),
#     "mmlu":                df.loc[df["type"] == "mmlu",                "category"].unique(),
#     "programming":         df.loc[df["type"] == "programming",         "category"].unique(),
# }


DATASET_TYPES = [
                "empathetic_dialogue", 
                "math", 
                "mmlu", 
                "programming"
                ]

print(DATASET_TYPES)

['empathetic_dialogue', 'math', 'mmlu', 'programming']


In [8]:
# skills = ["jurisprudence", "professional_law", "international_law", "us_foreign_policy", "security_studies"]
skills = ["philosophy", "moral_disputes", "moral_scenarios", "world_religions"]

# Which skills actually appear in df['category']?
present = set(df['category'].unique())
missing = [s for s in skills if s not in present]

if not missing:
    print("✅ All skills are present in df['category'].")
else:
    print("❌ Missing skills:", missing)

# df["category"].unique()

✅ All skills are present in df['category'].


In [9]:
import itertools
import pandas as pd

# --- your settings ---
dataset = "mmlu"
# skills = [
#     "afraid", "angry", "annoyed", "anxious", "ashamed", 
#     "devastated", "disappointed", "disgusted", "embarrassed", 
#     "furious", "guilty", "jealous", "lonely", "sad", "terrified"
# ]

# skills = ["caring", "confident", "content", "excited", "faithful", "grateful", "hopeful", "impressed", "joyful", "prepared", "proud", "surprised", "trusting"]
# skills = ["jurisprudence", "professional_law", "international_law", "us_foreign_policy", "security_studies"]
skills = ["philosophy", "moral_disputes", "moral_scenarios", "world_religions"]

TYPES = ["res", "mlp", "att"]
LAYERS = list(range(26))

results_list = []

# --- main loops ---
for typ in TYPES:
    for layer in LAYERS:
        col = (
            f"gemma-scope-2b-pt-{typ}-canonical-layer_{layer}"
            "/width_16k/canonical-token_feature_ids"
        )

        # for every unordered pair of skills
        for skill1, skill2 in itertools.combinations(skills, 2):
            # 1. filter each skill’s rows
            df1 = df[
                (df["type"] == dataset) &
                (df["category"] == skill1)
            ]
            df2 = df[
                (df["type"] == dataset) &
                (df["category"] == skill2)
            ]

            # 2. turn into vectors
            vec1 = to_vector(df1, col, binary_vector=False)
            vec2 = to_vector(df2, col, binary_vector=False)

            # 3. compute element‐wise product
            contribution_df = get_element_wise_product(
                typ, layer,
                skill1, skill2,
                vec1, vec2
            )

            # 4. collect non‐empty results
            if not contribution_df.empty:
                results_list.append(contribution_df)

In [8]:
# # 2 SKILLS 

# # DATASET_NAMES = ["emotion", "math", "mmlu", "programming"]
# import itertools
# import pandas as pd 
# # for type/ layer  
# # c1 
# # c2 

# TYPE = [
#         "res", 
#         "mlp", 
#         "att",
#        ]
# LAYER = [i for i in range(26)]
# results_list = []

# # your params
# class1, skill1 = "empathetic_dialogue", "sad"
# class2, skill2 = "empathetic_dialogue", "anxious"
# # class1 = "empathetic_dialogue"
# # skills = ["afraid", "angry", "annoyed", "anxious", "ashamed", "devastated", "disappointed", "disgusted", "embarrassed", "furious", "guilty", "jealous", "lonely", "sad", "terrified"]

# for typ in TYPE:
#     for layer in LAYER:
#         # between class
#         # for class1, class2 in itertools.combinations(skills, 2):
#             # for class1 in DATASET_TYPES:
#             #     for class2 in DATASET_TYPES:
#             #         if class1 < class2:
#             #             continue
#         col = f"gemma-scope-2b-pt-{typ}-canonical-layer_{layer}/width_16k/canonical-token_feature_ids"
        
#         # 1. get df 
#         class1_filtered = df[
#             (df['type'] == class1) &
#             (df['category'] == skill1)
#         ]
        
#         class2_filtered = df[
#             (df['type'] == class2) &
#             (df['category'] == skill2)
#         ]
        
#         # 2. to vector
#         vec_class1 = to_vector(class1_filtered, col, binary_vector=False)
#         vec_class2 = to_vector(class2_filtered, col, binary_vector=False)

#         print(typ, layer)

#         # return df : type layer class1 class2 contrib
#         # df += get_element_wise_product(typ, layer, class1, class2, vec_class1, vec_class2)

#         # 3. Calculate the element-wise product and get the results as a DataFrame
#         contribution_df = get_element_wise_product(typ, layer, skill1, skill2, vec_class1, vec_class2)
        
#         # 4. Append the resulting DataFrame to our list
#         if not contribution_df.empty:
#             results_list.append(contribution_df)

res 0
res 1
res 2
res 3
res 4
res 5
res 6
res 7
res 8
res 9
res 10
res 11
res 12
res 13
res 14
res 15
res 16
res 17
res 18
res 19
res 20
res 21
res 22
res 23
res 24
res 25
mlp 0
mlp 1
mlp 2
mlp 3
mlp 4
mlp 5
mlp 6
mlp 7
mlp 8
mlp 9
mlp 10
mlp 11
mlp 12
mlp 13
mlp 14
mlp 15
mlp 16
mlp 17
mlp 18
mlp 19
mlp 20
mlp 21
mlp 22
mlp 23
mlp 24
mlp 25
att 0
att 1
att 2
att 3
att 4
att 5
att 6
att 7
att 8
att 9
att 10
att 11
att 12
att 13
att 14
att 15
att 16
att 17
att 18
att 19
att 20
att 21
att 22
att 23
att 24
att 25


In [8]:
class1 = "programming"
skills = df["category"].unique()
print(skills)

['hard' 'easy' 'external' 'harder' 'hardest' 'medium' 'codeforces']


In [9]:
# FOR MANY CLASSES BETWEEN CLASS

import itertools
from tqdm import tqdm
import pandas as pd

# --- your settings ---
TYPES = ["res", "mlp", "att"]
LAYERS = list(range(26))

results_list = []

# for typ in TYPES:
for typ in tqdm(TYPES, desc="Type"):
    # Wrap the LAYERS loop, leave=False so it doesn’t linger when done
    # for layer in LAYERS:
    for layer in tqdm(LAYERS, desc="Layer", leave=False):
        col = (
            f"gemma-scope-2b-pt-{typ}-canonical-"
            f"layer_{layer}/width_16k/canonical-token_feature_ids"
        )

        for skill1, skill2 in itertools.combinations(skills, 2):
            # 1. filter for each skill within the same class
            df1 = df[
                (df["type"] == class1) &
                (df["category"] == skill1)
            ]
            df2 = df[
                (df["type"] == class1) &
                (df["category"] == skill2)
            ]

            # 2. turn each into a vector
            vec1 = to_vector(df1, col, binary_vector=False)
            vec2 = to_vector(df2, col, binary_vector=False)

            # 3. get element‐wise contributions
            contrib_df = get_element_wise_product(
                typ, layer, skill1, skill2, vec1, vec2
            )
            if contrib_df is None or contrib_df.empty:
                continue
            results_list.append(contrib_df)


Type: 100%|██████████| 3/3 [2:03:52<00:00, 2477.36s/it][A


In [10]:
# Concatenate all the individual result DataFrames into one large DataFrame
print("SAVE DATASET =======================================================")
final_df = pd.concat(results_list, ignore_index=True)

final_df = (
    final_df
    .sort_values('Contribution', ascending=False)
    .reset_index(drop=True)
)

# Save the final DataFrame to a CSV file
# path = f"./datasets/{skill1}_vs_{skill2}.csv"
# path = f"./datasets/dot_product/{class1}_between_class.csv"
# path = f"./datasets/dot_product/law_and_policy.csv"
path = f"./datasets/dot_product/philosophy_and_ethics.csv"
final_df.to_csv(path, index=False)

print(f"Successfully saved all neuron contributions to '{path}'")
print("\nFinal DataFrame head:")
print(final_df.head())

SAVE DATASET =======================================================
Successfully saved all neuron contributions to './datasets/dot_product/philosophy_and_ethics.csv'

Final DataFrame head:
  type  layer          class1           class2  Neuron_ID  Contribution
0  mlp     24  moral_disputes  moral_scenarios        282    10771296.0
1  mlp     24  moral_disputes  moral_scenarios       9329    10638350.0
2  att     12  moral_disputes  moral_scenarios       2424     9586056.0
3  mlp      7  moral_disputes  moral_scenarios       6117     9477090.0
4  att      5  moral_disputes  moral_scenarios       9670     9419795.0


# Remove duplicates 

In [11]:
# REMOVE DUPLICATES 
# path = f"./datasets/dot_product/{class1}_between_class.csv"
df = pd.read_csv(path)

df = (
    df
    .sort_values("Contribution", ascending=False)  # largest→smallest
    .drop_duplicates(
        subset=["type", "layer", "Neuron_ID"],
        keep="first"                               # now keeps the *largest* Contribution
    )
    .reset_index(drop=True)
)
# path = f"./datasets/dot_product/{class1}_between_class_clean.csv"
df.to_csv(path, index=False)

# Percentage 

In [12]:
# path = "./scratch/content_vs_hopeful.csv"
# path = f"./datasets/{skill1}_vs_{skill2}.csv"
# path = f"./datasets/negative_feelings.csv"

import pandas as pd
df = pd.read_csv(path, encoding="utf-8-sig")
df.head()

,type,layer,class1,class2,Neuron_ID,Contribution
0,mlp,24,devastated,embarrassed,282,12699340.0
1,att,22,devastated,embarrassed,14953,12099597.0
2,att,12,devastated,embarrassed,2424,10867400.0
3,mlp,24,embarrassed,furious,9329,10860906.0
4,att,7,ashamed,embarrassed,10364,10214085.0


In [12]:
import pandas as pd

def get_X_contribution(p):
    """
    Returns (idx, contribution) at percentile p through the DESC-sorted list.
    p should be between 0 and 1.
    """
    N = df.shape[0]
    idx = int(p * (N - 1))
    return idx, df.iloc[idx]["Contribution"]

topK = [0.01, 0.05 , 0.10, 0.12, 0.15, 0.18, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90]

idx_list = []
percentage_list = []
contrib_list = []

for p in topK:
    idx, contrib = get_X_contribution(p)
    percentage_list.append(p)
    idx_list.append(idx)
    contrib_list.append(contrib)

# now build a DataFrame
pct_df = pd.DataFrame({
    "Percentage": percentage_list,
    "Data point":    idx_list,
    "Contribution": contrib_list
})

print(pct_df)



    Percentage  Data point  Contribution
0         0.01        6390    10494080.0
1         0.05       31952      372186.0
2         0.10       63905       67980.0
3         0.12       76686       40984.0
4         0.15       95857       21660.0
5         0.18      115029       12464.0
6         0.20      127810        8892.0
7         0.30      191715        2200.0
8         0.40      255620         700.0
9         0.50      319526         252.0
10        0.60      383431          95.0
11        0.70      447336          35.0
12        0.80      511241          12.0
13        0.90      575146           4.0


In [ ]:
import pandas as pd

path = f"./datasets/dot_product/positive_felings.csv"

df = pd.read_csv(path)
# compute how many rows = 15%
# n = int(0.15 * len(df))

# take the first n rows
# top15pct = df.iloc[:n]
# print(f"Taking {n} rows out of {len(df)} total → {len(df)} rows")
top15pct = df.iloc[:50000]

print(top15pct.head())
print(top15pct.shape)

path = f"./datasets/dot_product/positive_felings.csv"
top15pct.to_csv(path, index=False)


In [14]:
# import pandas as pd

# classes = ["empathetic_dialogue", "math", "mmlu", "programming"]

# for cls in classes:
#     path = f"./datasets/dot_product/{cls}_between_class_clean.csv"

#     df = pd.read_csv(path)
#     # compute how many rows = 15%
#     # n = int(0.15 * len(df))
    
#     # take the first n rows
#     # top15pct = df.iloc[:n]
#     # print(f"Taking {n} rows out of {len(df)} total → {len(df)} rows")
#     top15pct = df.iloc[:50000]
    
#     print(top15pct.head())
#     print(top15pct.shape)
    
#     path = f"./datasets/dot_product/{cls}_between_class_clean_topk.csv"
#     top15pct.to_csv(path, index=False)

  type  layer        class1    class2  Neuron_ID  Contribution
0  mlp     24  apprehensive  trusting        282    15015042.0
1  att     22  apprehensive  trusting      14953    13710207.0
2  mlp     24  apprehensive  trusting       9329    12911080.0
3  att     12  apprehensive  trusting       2424    12815097.0
4  att      7  apprehensive  trusting      10364    11636983.0
(50000, 6)
  type  layer       class1      class2  Neuron_ID  Contribution
0  mlp      7  precalculus    geometry       6117  4.830805e+09
1  mlp     24  precalculus    geometry       9329  4.825307e+09
2  mlp     23  precalculus    geometry      10184  4.382280e+09
3  res      1  precalculus    geometry      12054  3.802484e+09
4  att      4     geometry  prealgebra       4742  3.458361e+09
(50000, 6)
  type  layer                        class1                     class2  \
0  mlp      7  high_school_european_history  high_school_world_history   
1  mlp     23  high_school_european_history  high_school_world_histo

# Auto interpert 

In [6]:
import pandas as pd
# path = "./datasets/negative_feelings.csv"
classes = ["empathetic_dialogue", "math", "mmlu", "programming"]
path = f"./datasets/dot_product/{classes[0]}_between_class_clean_topk.csv"
df = pd.read_csv(path, encoding="utf-8-sig")
df.head(20)

,type,layer,class1,class2,Neuron_ID,Contribution,Explanation
0,mlp,7,external,codeforces,6117,6.946736e+11,topics related to nutrition and health measur...
1,mlp,23,external,codeforces,10184,6.392421e+11,mathematical or analytical expressions related...
2,mlp,24,external,codeforces,9329,6.118761e+11,legal terms and phrases related to court case...
3,res,1,external,codeforces,12054,5.498718e+11,content related to medical conditions and trea...
4,mlp,4,external,codeforces,2793,4.683499e+11,terms related to legal and regulatory document...
5,mlp,24,external,codeforces,282,4.611576e+11,hyperlinks and web addresses
6,att,4,external,codeforces,4742,4.099533e+11,attends to specific keywords marked with doub...
7,mlp,4,external,codeforces,14164,3.239907e+11,structured programming elements and documentat...
8,mlp,25,external,codeforces,10593,2.584844e+11,references to personal experiences and societ...
9,res,25,external,codeforces,13749,2.384867e+11,specific references to medical and clinical tr...


In [7]:
import requests

def get_feature(model_id, source, index):
    try:
        url = f"https://www.neuronpedia.org/api/feature/{model_id}/{source}/{index}"
        resp = requests.get(url)
        resp.raise_for_status()            # throws if not 200
        feature = resp.json()
        
        # 2. Grab the list of explanations (might be empty!)
        explanations = feature.get("explanations", [])
        
        # 3. first description:
        # if explanations:
        return explanations[0]["description"]
    except Exception as e:
        return None

In [ ]:
from tqdm.auto import tqdm

model = "gemma-2-2b"
explanations = []
df["Explanation"] = None


# for idx, row in df.head(24700).iterrows():
# for idx, row in tqdm(df.iterrows(), total=len(df), desc="Fetching features"):
for idx, row in tqdm(df.head(24700).iterrows(), total=min(24700, len(df)), desc="Fetching features"):
    sae_type  = row["type"]
    layer     = row["layer"]
    neuron_id = row["Neuron_ID"]
    source    = f"{layer}-gemmascope-{sae_type}-16k"

    feat = get_feature(model, source, neuron_id)
    df.at[idx, "Explanation"] = feat
    # explanations.append(feat)

# df["Explanation"] = explanations
df.to_csv(path, index=False, encoding="utf-8-sig")

Fetching features:   0%|          | 0/24700 [00:00<?, ?it/s]

In [ ]:
sad sad 
sad anxious 


In [11]:
import pandas as pd
import asyncio
import aiohttp
from tqdm.asyncio import tqdm

model = "gemma-2-2b"
if "Explanation" not in df.columns:
    df["Explanation"] = None

chunk = df.iloc[0:25000]

# --- Define async API call ---
async def fetch_feature(session, idx, row):
    sae_type = row["type"]
    layer = row["layer"]
    neuron_id = row["Neuron_ID"]
    explanation = row["Explanation"]
    if pd.notna(explanation):
        return idx, explanation
    source = f"{layer}-gemmascope-{sae_type}-16k"
    url = f"https://www.neuronpedia.org/api/feature/{model}/{source}/{neuron_id}"

    try:
        async with session.get(url, timeout=10) as resp:
            data = await resp.json()
            explanations = data.get("explanations", [])
            return idx, explanations[0]["description"] if explanations else None
    except Exception as e:
        return idx, None

# --- Async executor ---
async def run():
    connector = aiohttp.TCPConnector(limit=1)  # Controls concurrency level
    timeout = aiohttp.ClientTimeout(total=10)
    async with aiohttp.ClientSession(connector=connector, timeout=timeout) as session:
        tasks = [fetch_feature(session, idx, row) for idx, row in chunk.iterrows()]
        results = []
        for f in tqdm.as_completed(tasks, total=len(tasks), desc="Async fetching"):
            result = await f
            results.append(result)
        return results

# --- Run & apply results ---
results = await run()
for idx, explanation in results:
    df.at[idx, "Explanation"] = explanation

df.to_csv(path, index=False)

Async fetching: 100%|██████████| 25000/25000 [00:12<00:00, 2008.30it/s]


In [12]:
df.iloc[2495:2500]

,type,layer,class1,class2,Neuron_ID,Contribution,Explanation
2495,att,21,external,codeforces,8769,68010708.0,None
2496,res,21,external,codeforces,11346,67939326.0,None
2497,res,0,external,codeforces,2896,67936050.0,None
2498,res,18,external,codeforces,11250,67930604.0,None
2499,att,1,external,codeforces,2935,67895360.0,None
